<a href="https://colab.research.google.com/github/talhaanwarch/schizophrenia64/blob/master/classification_1_all.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.ncbi.nlm.nih.gov/pubmed/21752709

In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [2]:
!pip install --upgrade numba
!pip install git+https://github.com/forrestbao/pyeeg.git
!pip install git+https://github.com/raphaelvallat/entropy.git


     |████████████████████████████████| 3.1MB 5.5MB/s 
     |████████████████████████████████| 24.6MB 2.1MB/s 
  Found existing installation: llvmlite 0.31.0
    Uninstalling llvmlite-0.31.0:
      Successfully uninstalled llvmlite-0.31.0
  Found existing installation: numba 0.48.0
    Uninstalling numba-0.48.0:
      Successfully uninstalled numba-0.48.0
  Cloning https://github.com/forrestbao/pyeeg.git to /tmp/pip-req-build-g1wm21wm
  Running command git clone -q https://github.com/forrestbao/pyeeg.git /tmp/pip-req-build-g1wm21wm
  Created wheel for pyeeg: filename=pyeeg-0.4.4-py2.py3-none-any.whl size=28122 sha256=f6f23cb64ac81c776eaff41bd8ef1eea3efe9c1e35b114ac892a0bca86bbae5b
  Stored in directory: /tmp/pip-ephem-wheel-cache-ho3f2m1y/wheels/2d/3f/ad/106d4fc80b61d1ea1fc18e76e7439fd98aa043d83d58eae741
Successfully built pyeeg
  Cloning https://github.com/raphaelvallat/entropy.git to /tmp/pip-req-build-2dsoiekm
  Running command git clone -q https://github.com/raphaelvallat/entropy.g

In [3]:
cd /content/drive/My Drive/dataset/Schizophrenia

/content/drive/My Drive/dataset/Schizophrenia


In [4]:
import numpy as np
from glob import glob

In [5]:
import pickle
import numpy as np
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MaxAbsScaler
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

In [6]:
from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from scipy import signal


# Read Data

In [7]:
ls

 demographic.csv         feature_acc_1_v1_manual.csv   feature_acc_3_v1.csv
 ERPdata.csv             feature_acc_2_pca.csv         feature_acc_v1.csv
'extended features'/     feature_acc_2_v1_manual.csv   pca.eps
 feature_acc_1_pca.csv   feature_acc_3_pca.csv         raw.csv


In [8]:
import pandas as pd
raw=pd.read_csv('raw.csv')

In [9]:

group=raw.groupby(['condition'])

data=group.get_group((1))





In [10]:
data=data.iloc[:,2::]
data.columns


Index(['Fp1', 'AF7', 'AF3', 'F1', 'F3', 'F5', 'F7', 'FT7', 'FC5', 'FC3', 'FC1',
       'C1', 'C3', 'C5', 'T7', 'TP7', 'CP5', 'CP3', 'CP1', 'P1', 'P3', 'P5',
       'P7', 'P9', 'PO7', 'PO3', 'O1', 'Iz', 'Oz', 'POz', 'Pz', 'CPz', 'Fpz',
       'Fp2', 'AF8', 'AF4', 'AFz', 'Fz', 'F2', 'F4', 'F6', 'F8', 'FT8', 'FC6',
       'FC4', 'FC2', 'FCz', 'Cz', 'C2', 'C4', 'C6', 'T8', 'TP8', 'CP6', 'CP4',
       'CP2', 'P2', 'P4', 'P6', 'P8', 'P10', 'PO8', 'PO4', 'O2', 'group'],
      dtype='object')

In [11]:
data.shape

(248832, 65)

In [12]:
label=data['group'].values
X=data.iloc[:,0:64].values


In [13]:
label=label.reshape(-1,3072)
label.shape

(81, 3072)

In [14]:
label=np.mean(label,axis=1)

In [15]:
ch=64
X_data=X.reshape(-1,3072,ch)
X_data.shape,label.shape

((81, 3072, 64), (81,))

# Feature Calculation

In [16]:
from scipy import stats
import pyeeg
from entropy import *
def mean(data):
    return np.mean(data,axis=0)
    
def std(data):
    return np.std(data,axis=0)

def ptp(data):
    return np.ptp(data,axis=0)

def var(data):
        return np.var(data,axis=0)

def minim(data):
      return np.min(data,axis=0)


def maxim(data):
      return np.max(data,axis=0)

def argminim(data):
      return np.argmin(data,axis=0)


def argmaxim(data):
      return np.argmax(data,axis=0)

def mean_square(data):
      return np.mean(data**2,axis=0)

def rms(data): #root mean square
      return  np.sqrt(np.mean(data**2,axis=0))  

def abs_diffs_signal(data):
    return np.sum(np.abs(np.diff(data,axis=0)),axis=0)


def skewness(data):
    return stats.skew(data,axis=0)

def kurtosis(data):
    return stats.kurtosis(data,axis=0)

def zero_crossing(data):
    return np.argmax(np.diff(np.sign(data),axis=0),axis=0)

def app_epy(data):
    result=[]
    for i in data.T:
        result.append(app_entropy(i, order=2, metric='chebyshev'))
    return np.array(result)

def perm_epy(data):
    result=[]
    for i in data.T:
        result.append(perm_entropy(i, order=3, normalize=True))
    return np.array(result)

def svd_epy(data):
    result=[]
    for i in data.T:
        result.append(svd_entropy(i, order=3, delay=1, normalize=True))
    return np.array(result)

def spectral_epy(data):
    result=[]
    for i in data.T:
        result.append(spectral_entropy(i, 100, method='welch', normalize=True))
    return np.array(result)

def sample_epy(data):
    result=[]
    for i in data.T:
        result.append(sample_entropy(i, order=2, metric='chebyshev'))
    return np.array(result)


def katz(data):
    result=[]
    for i in data.T:
        result.append(katz_fd(i))
    return np.array(result)

def higuchi(data):
    result=[]
    for i in data.T:
        result.append(higuchi_fd(i))
    return np.array(result)


def petrosian(data):
    result=[]
    for i in data.T:
        result.append(petrosian_fd(i))
    return np.array(result)

def autogressiveModelParameters(data):
    feature = []
    for i in data.T:
        coeff, sig = alg.AR_est_YW(i, order=5)
        feature.append(np.mean(coeff))
    return np.array(feature)

def teager(x):
    for i in range(len(x)-1):
        return x[i]**2 - (x[i-1]*x[i+1])


def lziv_complex(data):
    result=[]
    for i in data.T:
        result.append(lziv_complexity(i))
    return np.array(result)

        
def hjorth_mobility(data):
    return np.divide(np.std(np.diff(data,axis=0),axis=0),np.std(data,axis=0))

def hjorth_complexity(data):
    return np.divide(hjorth_mobility(np.diff(data,axis=0)),  hjorth_mobility(data))      




def concatenate_features(data):
    return np.concatenate((mean(data),std(data),ptp(data),var(data),minim(data),maxim(data),argminim(data),argmaxim(data),
                          mean_square(data),rms(data),abs_diffs_signal(data),
                          skewness(data),kurtosis(data),zero_crossing(data),
                          app_epy(data),perm_epy(data),svd_epy(data),spectral_epy(data),sample_epy(data),
                          katz(data),higuchi(data),petrosian(data),teager(data),lziv_complex(data),
                          hjorth_mobility(data),hjorth_complexity(data)),axis=0)

In [17]:
X_var=X_data.copy()
ch

64

In [18]:
from scipy import signal
nyq = 0.5 * 1024
l=0.1
low = l / nyq
high = 60/ nyq
b, a = signal.butter(3, [low,high], 'band')

In [19]:
features=[]
for i in range(len(X_var)):
    data=X_var[i,:,:]
    data=signal.filtfilt(b, a, data.T).T
    data=data[1300:1700,:]
    features.append(concatenate_features(data))
    



In [20]:
feature=np.array(features)
from sklearn.preprocessing import scale,maxabs_scale
feature=scale(feature)
feature.shape,label.shape

((81, 1664), (81,))

In [21]:
# dem=pd.read_csv('demographic.csv')
# print(dem.head())
# from sklearn import preprocessing
# le = preprocessing.LabelEncoder()
# dem[' gender']=le.fit_transform(dem[' gender'])
# print(dem.head())
# non_eeg=dem[[' gender', ' age']].values
# feature=np.concatenate((feature,non_eeg),1)

In [22]:



from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
def svc_param_selection(X, y):
    Cs = [ 0.1, 1, 10,15,20,25,30,40,50,60,70,100,120]
    gammas = [0.0001,0.0005,0.001,0.005,0.008, 0.01, 0.1,0.3,0.5, 1]
    param_grid = {'C': Cs, 'gamma' : gammas}
    grid_search = GridSearchCV(SVC(kernel='rbf'), param_grid, cv=10)
    grid_search.fit(X, y)
    grid_search.best_params_
    return grid_search.best_score_


svc_param_selection(feature,label)




0.7138888888888889

In [23]:
def dtree_param_selection(X,y):
    #create a dictionary of all values we want to test
    param_grid = { 'criterion':['gini','entropy'],'max_features':["auto", "sqrt", "log2"],'max_depth': np.arange(2, 20),'random_state':[10,20,30,40,50]}
    # decision tree model
    dtree_model=DecisionTreeClassifier()
    #use gridsearch to test all values
    dtree_gscv = GridSearchCV(dtree_model, param_grid, cv=10)
    #fit model to data
    dtree_gscv.fit(X, y)
    #print(dtree_gscv.best_score_)
    #print(dtree_gscv.best_estimator_)
    return dtree_gscv.best_score_
print(dtree_param_selection(feature,label))

0.6541666666666666


In [24]:

feature_list=['mean','std','ptp','var','minim','maxim','argmin','argmax','mean_square','rms','abs_diffs_signal','skewness','kurtosis','zero_crossing',
'app_epy','perm_epy','svd_epy','spectral_epy','sample_epy','katz','higuchi','petrosian','teager','lziv_complex',
'hjorth_mobility','hjorth_complexity']

In [25]:
feature_selected=[]
feat_non_sorted=[]
score_non_sorted=[]
for i ,j in zip(range(0,feature.shape[1],ch),feature_list):
  acc=svc_param_selection(feature[:,i:i+ch],label)
  print(j," : ",acc)
  feat_non_sorted.append(j)
  score_non_sorted.append(acc)
  #if acc>0.67:
   # feature_selected.append(j)


mean  :  0.7194444444444444
std  :  0.7180555555555556
ptp  :  0.7277777777777777
var  :  0.6666666666666666
minim  :  0.6541666666666666
maxim  :  0.7402777777777778
argmin  :  0.6319444444444444
argmax  :  0.6819444444444445
mean_square  :  0.6055555555555555
rms  :  0.6305555555555555
abs_diffs_signal  :  0.6402777777777777
skewness  :  0.6055555555555555
kurtosis  :  0.6652777777777777
zero_crossing  :  0.6430555555555555
app_epy  :  0.6777777777777778
perm_epy  :  0.6277777777777778
svd_epy  :  0.6055555555555555
spectral_epy  :  0.6402777777777777
sample_epy  :  0.6416666666666666
katz  :  0.6416666666666666
higuchi  :  0.6069444444444445
petrosian  :  0.6166666666666666
teager  :  0.6305555555555555
lziv_complex  :  0.6569444444444444
hjorth_mobility  :  0.6541666666666666
hjorth_complexity  :  0.6305555555555555


In [26]:

print(feat_non_sorted)
print(score_non_sorted)

['mean', 'std', 'ptp', 'var', 'minim', 'maxim', 'argmin', 'argmax', 'mean_square', 'rms', 'abs_diffs_signal', 'skewness', 'kurtosis', 'zero_crossing', 'app_epy', 'perm_epy', 'svd_epy', 'spectral_epy', 'sample_epy', 'katz', 'higuchi', 'petrosian', 'teager', 'lziv_complex', 'hjorth_mobility', 'hjorth_complexity']
[0.7194444444444444, 0.7180555555555556, 0.7277777777777777, 0.6666666666666666, 0.6541666666666666, 0.7402777777777778, 0.6319444444444444, 0.6819444444444445, 0.6055555555555555, 0.6305555555555555, 0.6402777777777777, 0.6055555555555555, 0.6652777777777777, 0.6430555555555555, 0.6777777777777778, 0.6277777777777778, 0.6055555555555555, 0.6402777777777777, 0.6416666666666666, 0.6416666666666666, 0.6069444444444445, 0.6166666666666666, 0.6305555555555555, 0.6569444444444444, 0.6541666666666666, 0.6305555555555555]


In [27]:
score,feat  = zip(*sorted(zip(score_non_sorted, feat_non_sorted),reverse=True))

In [28]:
print(feat)
print(score)

('maxim', 'ptp', 'mean', 'std', 'argmax', 'app_epy', 'var', 'kurtosis', 'lziv_complex', 'minim', 'hjorth_mobility', 'zero_crossing', 'sample_epy', 'katz', 'spectral_epy', 'abs_diffs_signal', 'argmin', 'teager', 'rms', 'hjorth_complexity', 'perm_epy', 'petrosian', 'higuchi', 'svd_epy', 'skewness', 'mean_square')
(0.7402777777777778, 0.7277777777777777, 0.7194444444444444, 0.7180555555555556, 0.6819444444444445, 0.6777777777777778, 0.6666666666666666, 0.6652777777777777, 0.6569444444444444, 0.6541666666666666, 0.6541666666666666, 0.6430555555555555, 0.6416666666666666, 0.6416666666666666, 0.6402777777777777, 0.6402777777777777, 0.6319444444444444, 0.6305555555555555, 0.6305555555555555, 0.6305555555555555, 0.6277777777777778, 0.6166666666666666, 0.6069444444444445, 0.6055555555555555, 0.6055555555555555, 0.6055555555555555)


In [29]:
def del_item(a, b):
     return [x for x in a if x not in b]



#skf = StratifiedKFold(n_splits=10, random_state=2020, shuffle=False)
acc=0
deleted_item=[]
for i in range(1,20):
  feature_selected=list(feat[:i])
  # feature_selected=del_item(feature_selected,deleted_item)

  X_good=[]
  for key,val in zip(feature_list,range(0,feature.shape[1],ch)):
      for fe in feature_selected:
          if key==fe:     
              #print('key',key,'value',val,":",val+ch) 
              X_good.append(feature[:,val:val+ch])
  good_feature=np.concatenate((X_good),axis=1)
  #good_feature=np.concatenate((good_feature,non_eeg),1)


  acc_new=svc_param_selection(good_feature,label)
  print(i,' : ', acc_new)

  # print(acc_new, good_feature.shape)
  # if acc_new<(acc-0.01):
  #   deleted_item.append(feature_selected.pop())
  #   print('del item',deleted_item)
  # else:
  #   acc=acc_new


1  :  0.7402777777777778
2  :  0.7027777777777777
3  :  0.7138888888888889
4  :  0.6944444444444444
5  :  0.763888888888889
6  :  0.7375
7  :  0.75
8  :  0.7402777777777778
9  :  0.7513888888888889
10  :  0.763888888888889
11  :  0.7527777777777778
12  :  0.7375
13  :  0.7375
14  :  0.725
15  :  0.7125
16  :  0.7125
17  :  0.7138888888888889
18  :  0.7152777777777778
19  :  0.726388888888889


In [ ]:
# Result Section

In [30]:
feature_selected=feat[:5]
X_good=[]
for key,val in zip(feature_list,range(0,feature.shape[1],ch)):
    for fe in feature_selected:
        if key==fe:     
            print('key',key,'value',val,":",val+ch) 
            X_good.append(feature[:,val:val+ch])

key mean value 0 : 64
key std value 64 : 128
key ptp value 128 : 192
key maxim value 320 : 384
key argmax value 448 : 512


In [31]:
good_feature=np.concatenate((X_good),axis=1)
good_feature.shape

(81, 320)

In [32]:
def svc_param_selection(X, y, nfolds):
    Cs = [ 0.1, 1, 10,15,20,25,30,40,50,60,70,80,100,120]
    gammas = [0.0001,0.0005,0.001,0.005,0.008, 0.01, 0.1,0.3,0.5, 1]
    param_grid = {'C': Cs, 'gamma' : gammas}
    grid_search = GridSearchCV(SVC(kernel='rbf'), param_grid, cv=nfolds)
    grid_search.fit(X, y)
    print(grid_search.best_score_)
    return grid_search.best_params_


svc_param_selection(good_feature,label,10)

0.763888888888889


{'C': 1, 'gamma': 0.01}

In [ ]:
clf = SVC(kernel="rbf", C=1,gamma=0.01, probability=True)
acc = cross_val_score(clf,good_feature,label,cv=10)
precision = cross_val_score(clf,good_feature,label,cv=10,scoring='precision')
recall = cross_val_score(clf,good_feature,label,cv=10,scoring='recall')
f1 = cross_val_score(clf,good_feature,label,cv=10,scoring='f1')

print('average accuracy : ',np.array(acc).mean(),np.std(np.array(acc)))
print('average precision : ',np.array(precision).mean(),np.std(np.array(precision)))
print('average recall : ',np.array(recall).mean(),np.std(np.array(recall)))
print('average f1 score : ',np.array(f1).mean(),np.std(np.array(f1)))

average accuracy :  0.763888888888889 0.10540925533894598
average precision :  0.7595238095238095 0.07708551660816902
average recall :  0.9 0.1341640786499874
average f1 score :  0.8193939393939393 0.08611820522334823


In [ ]:
def knn_param_selection(X, y, nfolds):
    n_neighbors  = [ 3,4,5,6,7,8,9]
    weights  = ['uniform','distance']
    metric=['minkowski','manhattan','euclidean']
    param_grid = {'n_neighbors': n_neighbors, 'weights' : weights,'metric':metric}
    grid_search =GridSearchCV( KNeighborsClassifier(), param_grid, cv=nfolds,n_jobs=-1)
    grid_search.fit(X, y)
    print(grid_search.best_score_)
    return grid_search.best_params_


knn_param_selection(good_feature,label,10)

0.6902777777777778


{'metric': 'minkowski', 'n_neighbors': 5, 'weights': 'uniform'}

In [ ]:
clf = KNeighborsClassifier(metric= 'minkowski', n_neighbors=5, weights= 'uniform')
acc = cross_val_score(clf,good_feature,label,cv=10)
precision = cross_val_score(clf,good_feature,label,cv=10,scoring='precision')
recall = cross_val_score(clf,good_feature,label,cv=10,scoring='recall')
f1 = cross_val_score(clf,good_feature,label,cv=10,scoring='f1')

print('average accuracy : ',np.array(acc).mean(),np.std(np.array(acc)))
print('average precision : ',np.array(precision).mean(),np.std(np.array(precision)))
print('average recall : ',np.array(recall).mean(),np.std(np.array(recall)))
print('average f1 score : ',np.array(f1).mean(),np.std(np.array(f1)))

average accuracy :  0.6902777777777778 0.11703809920990116
average precision :  0.7597619047619047 0.14768220615092884
average recall :  0.775 0.16620770138594662
average f1 score :  0.7451226551226551 0.10802239890385032


In [ ]:
from sklearn.linear_model import LogisticRegression
def logistic_param_selection(X, y, nfolds):
    C= [0.01,0.05,0.1,0.5, 1,2,3,4,5,8, 10,12,15]
    max_it= [200,300,400,500]
    param_grid = {'C': C,'max_iter':max_it}
    grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=nfolds)
    grid_search.fit(X, y)
    print(grid_search.best_score_)
    return grid_search.best_params_


logistic_param_selection(good_feature,label,10)

0.8388888888888889


{'C': 0.5, 'max_iter': 200}

In [ ]:
clf = LogisticRegression(max_iter=200,C=0.4)
acc = cross_val_score(clf,good_feature,label,cv=10)
precision = cross_val_score(clf,good_feature,label,cv=10,scoring='precision')
recall = cross_val_score(clf,good_feature,label,cv=10,scoring='recall')
f1 = cross_val_score(clf,good_feature,label,cv=10,scoring='f1')

print('average accuracy : ',np.array(acc).mean(),np.std(np.array(acc)))
print('average precision : ',np.array(precision).mean(),np.std(np.array(precision)))
print('average recall : ',np.array(recall).mean(),np.std(np.array(recall)))
print('average f1 score : ',np.array(f1).mean(),np.std(np.array(f1)))

average accuracy :  0.8388888888888889 0.1130388330520878
average precision :  0.8430952380952382 0.11505913889029794
average recall :  0.915 0.10499999999999998
average f1 score :  0.8726767676767677 0.08926643020574633


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold
#skf = StratifiedKFold(n_splits=10, random_state=2020, shuffle=False)
def dtree_param_selection(X,y):
    #create a dictionary of all values we want to test
    param_grid = { 'criterion':['gini','entropy'],'max_features':["auto", "sqrt", "log2"],'max_depth': np.arange(2, 20),'random_state':[10,20,30,40]}
    # decision tree model
    dtree_model=DecisionTreeClassifier()
    #use gridsearch to test all values
    dtree_gscv = GridSearchCV(dtree_model, param_grid, cv=10)
    #fit model to data
    dtree_gscv.fit(X, y)
    print(dtree_gscv.best_score_)
    #print(dtree_gscv.best_estimator_)
    return dtree_gscv.best_params_

dtree_param_selection(good_feature,label)

0.725


{'criterion': 'entropy',
 'max_depth': 2,
 'max_features': 'auto',
 'random_state': 10}

In [ ]:
clf = DecisionTreeClassifier(criterion= 'entropy', max_depth= 6, max_features= 'log2',random_state=40)
acc = cross_val_score(clf,good_feature,label,cv=10)
precision = cross_val_score(clf,good_feature,label,cv=10,scoring='precision')
recall = cross_val_score(clf,good_feature,label,cv=10,scoring='recall')
f1 = cross_val_score(clf,good_feature,label,cv=10,scoring='f1')

print('average accuracy : ',np.array(acc).mean(),np.std(np.array(acc)))
print('average precision : ',np.array(precision).mean(),np.std(np.array(precision)))
print('average recall : ',np.array(recall).mean(),np.std(np.array(recall)))
print('average f1 score : ',np.array(f1).mean(),np.std(np.array(f1)))

average accuracy :  0.7041666666666666 0.08003905296791061
average precision :  0.7351190476190476 0.0967605250417194
average recall :  0.8350000000000002 0.15173990905493517
average f1 score :  0.7695998445998445 0.07181482223892126


In [ ]:
df=pd.DataFrame(list(zip(feat_non_sorted,score_non_sorted)), columns=['Feature','Accuracy'])
df.to_csv('feature_acc_v1.csv')
